# sql语义分析
## 了解arrow
在讲解语义分析的过程中利用arrow的类型、接口进行演示。



## 基础数据结构
构建过程中的用到的数据结构。
- scope 和 env
- logicalPlan/substrait

## 语义分析
select语句各个部分的bind方法。

- 列引用的可见范围
- 聚合函数
- 分组
- 表达式

## 子查询

## CTE

## 产生逻辑计划
### tableScan

### join

### 聚合

### filter

### project

## 优化器
### 框架
### 数据结构
### 优化器规则

# 取 tpch q1 ast

In [450]:
from pglast import ast,parser,visitors,printers,enums
from pprint import pprint

In [451]:
q1Stmt = parser.parse_sql(
    "select \
        l_returnflag, \
        l_linestatus, \
        sum(l_quantity) as sum_qty, \
        sum(l_extendedprice) as sum_base_price, \
        sum(l_extendedprice * (1 - l_discount)) as sum_disc_price, \
        sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)) as sum_charge, \
        avg(l_quantity) as avg_qty, \
        avg(l_extendedprice) as avg_price, \
        avg(l_discount) as avg_disc, \
        count(*) as count_order \
    from \
        lineitem \
    where \
        l_shipdate <= date '1998-12-01' - interval ':1' day \
    group by \
        l_returnflag, \
        l_linestatus \
    order by \
        l_returnflag, \
        l_linestatus;"
)[0]

In [452]:
#q1Stmt

# 表lineitem的schema

In [453]:
import pyarrow as arrow

In [454]:
# create table lineitem ( l_orderkey    bigint not null,
#                              l_partkey     integer not null,
#                              l_suppkey     integer not null,
#                              l_linenumber  integer not null,
#                              l_quantity    decimal(15,2) not null,
#                              l_extendedprice  decimal(15,2) not null,
#                              l_discount    decimal(15,2) not null,
#                              l_tax         decimal(15,2) not null,
#                              l_returnflag  varchar(1) not null,
#                              l_linestatus  varchar(1) not null,
#                              l_shipdate    date not null,
#                              l_commitdate  date not null,
#                              l_receiptdate date not null,
#                              l_shipinstruct varchar(25) /*char(25)*/ not null,
#                              l_shipmode     varchar(10) /*char(10)*/ not null,
#                              l_comment      varchar(44) not null,
#                          primary key (l_orderkey, l_linenumber)
#                         );

In [455]:
lineitemSchema = arrow.schema([
    arrow.field("l_orderkey",arrow.int64(),False),
    arrow.field("l_partkey",arrow.int32(),False),
    arrow.field("l_suppkey",arrow.int32(),False),
    arrow.field("l_linenumber",arrow.int32(),False),
    arrow.field("l_quantity",arrow.decimal128(15,2),False),
    arrow.field("l_extendedprice",arrow.decimal128(15,2),False),
    arrow.field("l_discount",arrow.decimal128(15,2),False),
    arrow.field("l_tax",arrow.decimal128(15,2),False),
    arrow.field("l_returnflag",arrow.utf8(),False),
    arrow.field("l_linestatus",arrow.utf8(),False),
    arrow.field("l_shipdate",arrow.date32(),False),
    arrow.field("l_commitdate",arrow.date32(),False),
    arrow.field("l_receiptdate",arrow.date32(),False),
    arrow.field("l_shipinstruct",arrow.utf8(),False),
    arrow.field("l_shipmode",arrow.utf8(),False),
    arrow.field("l_comment",arrow.utf8(),False),
])
# lineitemSchema

In [456]:
Catalog = {
    "tpch":{
        "lineitem":lineitemSchema
    }
}
Catalog

{'tpch': {'lineitem': l_orderkey: int64 not null
l_partkey: int32 not null
l_suppkey: int32 not null
l_linenumber: int32 not null
l_quantity: decimal128(15, 2) not null
l_extendedprice: decimal128(15, 2) not null
l_discount: decimal128(15, 2) not null
l_tax: decimal128(15, 2) not null
l_returnflag: string not null
l_linestatus: string not null
l_shipdate: date32[day] not null
l_commitdate: date32[day] not null
l_receiptdate: date32[day] not null
l_shipinstruct: string not null
l_shipmode: string not null
l_comment: string not null}}

# 构建tpch q1的逻辑查询计划

## 逻辑查询计划builder

从各种SQL语句构建逻辑查询计划

逻辑查询计划定义为字典类型。

In [457]:
RELATION_TYPE_TABLE = 0
RELATION_TYPE_VIEW = 1
RELATION_TYPE_SUBQUERY = 2

class LogicalPlanBuilder:
    def __init__(self):
        pass

    def build(self,node,plan : dict):
        pass

## select语句builder

为select语句生成逻辑查询计划。

In [458]:
class SelectBuilder(LogicalPlanBuilder):
    def __init__(self):
        super().__init__()

    def build(self,select : ast.SelectStmt,plan : dict):
        fb = FromBuilder()
        fb.build(select.fromClause,plan)

        wb = WhereBuilder()
        wb.build(select.whereClause,plan)
        pass

## from子句builder

从from子句中提取各个关系表。

In [459]:
class FromBuilder(LogicalPlanBuilder):
    def __init__(self):
        super().__init__()
    def build(self,tableRefs : tuple,plan : dict):
        print("tableRefs")
        if len(tableRefs) == 1:
            single = self.buildTableRef(tableRefs[0])
            plan["relations"] = {"singleRelation":single}
            return
        raise Exception("unsupport multiple table refs")
        pass

    def buildTableRef(self,tableRef : ast.RangeVar)-> dict:
        print("tableRef")
        dbName = tableRef.schemaname
        if dbName is None:
            dbName = "tpch"

        #print(Catalog[dbName])
        if tableRef.relname in Catalog[dbName] :
            return {tableRef.relname :
                    ["relationRef", #label. relation reference
                    RELATION_TYPE_TABLE, #relation type
                    dbName, #database name
                    tableRef.relname,#alias=
                    tableRef.relname,#original name
                    Catalog[dbName][tableRef.relname] #schema
                    ]}
        else:
            raise Exception("no such table in Catalog",tableRef.schemaname,tableRef.relname)
        pass

## 表达式builder

表达式构建基类。也是最复杂的类。

In [460]:
class ExpressionBuilder(LogicalPlanBuilder):
    def __init__(self):
        super().__init__()

    def build(self,node,plan : dict):
        if isinstance(node,ast.A_Expr):
            if node.kind == enums.parsenodes.A_Expr_Kind.AEXPR_OP:
                opName = node.name[0].sval
                if opName == "<=":
                    self.build(node.lexpr,plan)
                    self.build(node.rexpr,plan)
                elif opName == "-":
                    self.build(node.lexpr,plan)
                    self.build(node.rexpr,plan)
                else:
                    raise Exception("unsupported operator",node)
            else:
                raise Exception("unsupported expr 1",node)
        elif isinstance(node,ast.ColumnRef):
            fields = node.fields
            if len(fields) == 1:
                colName = fields[0].sval
                print("==",colName)
            raise Exception("unsupported column ref",node)
        elif isinstance(node,ast.TypeCast):
            raise Exception("unsupported typeCast",node)
        elif isinstance(node,ast.A_Const):
            raise Exception("unsupported const",node)
        else:
            raise Exception("unsupported expr 2",node)
        pass

## where子句builder

构建where表达式。

In [461]:
class WhereBuilder(LogicalPlanBuilder):
    def __init__(self):
        super().__init__()

    def build(self,node,plan : dict):
        print("where ",node)
        eb = ExpressionBuilder()
        eb.build(node,plan)
        pass

In [462]:
plan = {}
selBuilder = SelectBuilder()
selBuilder.build(q1Stmt.stmt,plan)

tableRefs
tableRef
where  <A_Expr kind=<A_Expr_Kind.AEXPR_OP: 0> name=(<String sval='<='>,) lexpr=<ColumnRef fields=(<String sval='l_shipdate'>,)> rexpr=<A_Expr kind=<A_Expr_Kind.AEXPR_OP: 0> name=(<String sval='-'>,) lexpr=<TypeCast arg=<A_Const isnull=False val=<String sval='1998-12-01'>> typeName=<TypeName names=(<String sval='date'>,) setof=False pct_type=False typemod=-1>> rexpr=<TypeCast arg=<A_Const isnull=False val=<String sval=':1'>> typeName=<TypeName names=(<String sval='pg_catalog'>, <String sval='interval'>) setof=False pct_type=False typmods=(<A_Const isnull=False val=<Integer ival=8>>,) typemod=-1>>>>
== l_shipdate


Exception: ('unsupported column ref', <ColumnRef fields=(<String sval='l_shipdate'>,)>)